<div class="alert alert-block alert-success">
    <h1>DataLab HvA - serie Machine Learning Python - Tech report 11</h1>
</div>

<div class="alert alert-block alert-warning">
    <h2>Support Vector Machines - Regressie</h2>
</div>

### 0. Doel van deze les
Vanwege de serie lessen waarbinnen deze les valt is het hierin de insteek is om het gemaakte model te gaan vergelijken met andere andere Machine Learning modellen en daarbij een kwaliteitscriterium te hanteren waarmee *alle* modellen onderling vergelijkbaar zijn. Dit is de reden dat we werken met een test- en een trainset van onze data.
**Je leert** 
- hoe je in Python met behulp van het package `sklearn` een Support Vector Machine (SVM) t.b.v. een Regressietaak kunt maken (ook wel Support Vector Regressor - SVR)
- welke hyperparameters er gekozen kunnen worden om het model te optimaliseren
- hoe je het gevonden model kunt weergeven
- hoe je dit model kunt gebruiken om voorspellingen te genereren op nieuwe data

**Je leert niet:**
- wat de preciese wiskundige definitie en werking van het algoritme is
- hoe je met het package `seaborn` eerste visuele verkenningen van de (variatie in de) data kan uitvoeren, zie hiervoor *Tech report 1: Introductie Machine Learning* uit deze serie.
- hoe je data importeert en inlaadt in de python sessie, zie hiervoor echter wel [deze instructie](https://www.youtube.com/watch?v=1oBVx7pyuXo&list=PLwkTCAI_gJjW10xaAJpdqeQx7EQgFolfG&index=10) 
- wat het preciese concept en DE werking van K-fold crossvalidatie is, zie hiervoor *Tech report 6: K-fold crossvalidatie* uit deze serie.

**Machine Learning framework**
- Om een consistente leerlijn in de serie Machine Learning aan te houden kiezen we telkens voor een opzet waarin een inititeel model bepaald wordt o.b.v. een *deel* van de beschikbare data: de *train data*. Om het model te kunnen vergelijken met ándere modeltypen (waarbij andere algoritmen worden gebruikt) stellen we de voorspellende prestaties vast a.h.v. de nog niet gebruikte data: de *test data*. 

### 1. Laden van de benodigde packages
Packages die je zelf nog niet hebt geinstalleerd dien je vooraf in Python te installeren. 

In [1]:
# Voor de dataset en plots
from sklearn import datasets
import matplotlib.pyplot as plt
import seaborn as sns

# Voor het bewerken en inspecteren van de data
import pandas as pd

# Voor het afsplitsen van een train- en test dataset
from sklearn.model_selection import train_test_split

# Voor het modelleren
from sklearn import svm

### 2. Laden en prepareren van de data

We gebruiken als voorbeeld een dataset `'diamonds'`. Het betreft data van verschillende kenmerken uit een grote verzameling waarnemingen van diamanten. Eén van de kenmerken is de prijs, uitgedrukt in het aantal *US dollars* (`price`). De prijs zal onder andere afhangen van andere kenmerken die te maken hebben met de afmetingen (`x`, `y` en `z`), het gewicht (`carat`), de kwaliteit van het het slijpen (`cut`), de kleur (`color`) en de helderheid (`clarity`). Lees meer over deze dataset in de link [hier](https://ggplot2.tidyverse.org/reference/diamonds.html), daar worden ook de variabelen `table` en `depth` toegelicht.

#### Laden en weergave data
Zie de code hieronder voor het laden van de data. Er zijn geen missende waarden aanwezig (inspecteer dit ook zélf!) - zie *TR2 - Datapreparatie*  uit deze serie. 

In [2]:
# Laden van een standaard dataset uit package seaborn 
data = sns.load_dataset('diamonds')
print(data.head())
print(data.shape)
print(data.dtypes)

   carat      cut color clarity  depth  table  price     x     y     z
0   0.23    Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1   0.21  Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2   0.23     Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3   0.29  Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4   0.31     Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
(53940, 10)
carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
price         int64
x           float64
y           float64
z           float64
dtype: object


#### One-Hot encoding categoriale variabelen
We zien dat we te maken hebben met drie categoriale variabelen (`cut`, `color` en `clarity`), het random forest kan hier niet mee overweg. We maken hiervoor *one-hot encoded variabelen* aan, voor iedere waarde van de categoriale variabelen één (`drop_first=False`) :

In [3]:
data = pd.get_dummies(data=data, drop_first=False)
print(data.head())
print(data.shape)
print(data.dtypes)

   carat  depth  table  price     x     y     z  cut_Ideal  cut_Premium  \
0   0.23   61.5   55.0    326  3.95  3.98  2.43       True        False   
1   0.21   59.8   61.0    326  3.89  3.84  2.31      False         True   
2   0.23   56.9   65.0    327  4.05  4.07  2.31      False        False   
3   0.29   62.4   58.0    334  4.20  4.23  2.63      False         True   
4   0.31   63.3   58.0    335  4.34  4.35  2.75      False        False   

   cut_Very Good  ...  color_I  color_J  clarity_IF  clarity_VVS1  \
0          False  ...    False    False       False         False   
1          False  ...    False    False       False         False   
2          False  ...    False    False       False         False   
3          False  ...     True    False       False         False   
4          False  ...    False     True       False         False   

   clarity_VVS2  clarity_VS1  clarity_VS2  clarity_SI1  clarity_SI2  \
0         False        False        False        False         

#### Split variabelen X en y
In dit voorbeeld willen we de waarde voor `price` gaan voorspellen, zie ook *sectie 3: Voorbeschouwing* verderop.

In [4]:
#independent variables / explanatory variables
X = data.drop('price', axis=1)

#dependent variable / response / target variable.
y = data['price']

In [5]:
#type(y) #pandas.core.series.Series
type(X) #pandas.core.frame.DataFrame

pandas.core.frame.DataFrame

#### Afsplitsen test data en train data
Voor meer details over deze stap, zie *Tech report 2: Datapreparatie* uit deze serie.

In [6]:
# Split in training en test data 
# random_state kiezen zorgt voor reproduceerbaarheid van dit resultaat - gebruik dat ! Een andere waarde is uiteraard ook prima
# we kiezen voor een verhouden train - test van 70% - 30%. Andere keuzen kunnen ook!
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.30,random_state=999)
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

(37758, 26)
(16182, 26)
(37758,)
(16182,)


### 3. Voorbeschouwing
Zomaar modelleren doe je niet, eerst moeten we vaststellen wat we willen bereiken of welke vraag we zouden willen beantwoorden. Voor deze les zal dat worden:
- Taak: stel een random forest regressiemodel op om de hoogte van de prijs zo goed mogelijk te voorspellen op basis van de overige variabelen 

Vanuit de taak zullen we dus stellen:
- De *onafhankelijke* variabelen ($x_i$) zijn de variabelen `carat`, `cut_[5x]`, `color_[7x]`, `clarity_[8x]`, `depth`, `table`, `x`, `y`, en `z`.
- De *afhankelijke* variabele ($y$) is de variabele `price`.

#### Support Vector Machine - Regressie
Het algoritme is gebaseerd op het zoeken van zogenaamde *hyperplanes* in de *(getransformeerde - zie onder) feature space X*. De uiteindelijk gevonden optimale *hyperplanes* worden vastgelegd door een beperkt aantal *support vectors* (een door het algoritme gevonden optimale set waarnemingen uit de traindata *X*), hieraan ontleent dit algoritme zijn naam. Die hyperplanes worden zo gekozen in het trainingsproces dat een kostenfunctie - die vooral de complexiteit van het model 'bewaakt' en een penalty toekent aan voorspelfouten die té groot worden - zo klein mogelijk is. Die penalty op voorspelfouten blijft achterwege zolang deze een vooraf gegeven *epsilon* ($\epsilon$) niet overschrijden. Het gevolg is dat hoe **groter** deze *hyperparameter epsilon* gekozen is, hoe **minder** hyperplanes er nodig zullen zijn en dus hoe minder complex het model zal hoeven worden. De relatie tussen de support vectors en de voorspelling wordt hier verderop gegegeven. 

Een sterk punt is dat het algoritme juist heel goed overweg kan met verbanden *y ~ X* die in eerste instantie juist *niet* lineair zijn door de data *eerst* te transformeren naar een *hogere dimensie* waarin de data zich *beter* volgens een lineaire relatie *y ~ X* gedraagt. Die transformatie wordt de *kernel trick* genoemd. Hierbij verwijst de *kernel* naar een hyperparameter die dient te worden gekozen en de wijze waarop de data wordt getransformeerd bepaalt. Een ander sterk punt is dat zolang de voorspellingen van de traindata binnen de bandbreedte *epsilon* (de *epsilon tube*) blijven het model niet zal wijzigen, immers: dit beïnvloed de kostenfunctie niet. Dit maakt het model robuust m.b.t. variatie in traindata. 

Voor de wat meer wiskundig geïnteresseerden: het genereren van een *voorspelling* $f(x)$ op (nieuwe $x$) data kun je je als volgt voorstellen:

$$ f(x) = \beta_0 + \sum_{i \in \mathcal{S}} \alpha_i K(x, x_i)$$

Hierin is $\mathcal{S}$ de indexverzameling van waarnemingen uit de traindata die door het algoritme als de *support vectors* zijn aangewezen, de $\beta_0$ en $\alpha_i$ de gevonden modelcoëfficiënten en $K(\dots , \dots)$ het door de *kernelfunctie* voortgebrachte inproduct. 

Het SVM algoritme wordt wat uitvoeriger toegelicht op [deze](https://scikit-learn.org/stable/modules/svm.html#) pagina. 

### 4. Model schatten
#### lees documentatie bij package
Voordat we een model gaan schatten is het van belang dat je de documentatie van het algoritme dat we uit het package gaan gebruiken lezen. In dit geval is dat dus voor `svm.SVR()`, die lees je bijvoorbeeld online [hier](https://scikit-learn.org/stable/modules/svm.html#regression). Alternatief is om de `help()` functie te gebruiken, zie hieronder. 

In [7]:
# We gaan de methode svm.SVC() uit het sklearn package gebruiken, inspecteer hiervoor de documentatie. Dat kan 
help(svm.SVR())

Help on SVR in module sklearn.svm._classes object:

class SVR(sklearn.base.RegressorMixin, sklearn.svm._base.BaseLibSVM)
 |  SVR(*, kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
 |  
 |  Epsilon-Support Vector Regression.
 |  
 |  The free parameters in the model are C and epsilon.
 |  
 |  The implementation is based on libsvm. The fit time complexity
 |  is more than quadratic with the number of samples which makes it hard
 |  to scale to datasets with more than a couple of 10000 samples. For large
 |  datasets consider using :class:`~sklearn.svm.LinearSVR` or
 |  :class:`~sklearn.linear_model.SGDRegressor` instead, possibly after a
 |  :class:`~sklearn.kernel_approximation.Nystroem` transformer or
 |  other :ref:`kernel_approximation`.
 |  
 |  Read more in the :ref:`User Guide <svm_regression>`.
 |  
 |  Parameters
 |  ----------
 |  kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precompu

Zie bijvoorbeeld de parameters: `C`, `kernel`, `degree`, `gamma` en `epsilon`. Ook lees je dat er een *method*  `fit()` voor deze *object class* gedefinieerd is om het model te *fitten*.

#### Standaardiseren X-data
Voor een SVM is het aan te raden de numerieke variabelen van de onafhankelijke data te standaardiseren. Lees hierover meer in *Tech Report 2 - Datapreparatie* (sectie 4) uit deze serie. In de stappen hieronder doorlopen we dit proces en zal duidelijk worden dat de prestaties verbeterd zijn.

In [8]:
#Inladen method voor standaardiseren
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_X) #Let op: de scaler wordt gefit op de traindataset

#herschalen van de X-data uit de trainset
train_X_scaled = pd.DataFrame(scaler.transform(train_X), columns = X.columns)
print(train_X_scaled.head())

#herschalen van de X-data uit de testset - NB: op basis van de scaler die is gefit op de traindata!
test_X_scaled = pd.DataFrame(scaler.transform(test_X), columns = X.columns)


      carat     depth     table         x         y         z  cut_Ideal  \
0 -1.112647  0.390313 -0.653967 -1.400871 -1.338917 -1.322086  -0.815542   
1 -0.204115 -0.940945  1.588651 -0.043545  0.032078 -0.109070  -0.815542   
2 -1.028132 -0.030084  1.588651 -1.240135 -1.234791 -1.223353  -0.815542   
3 -1.028132 -1.151144  0.691604 -1.204416 -1.148019 -1.265667  -0.815542   
4  0.619902  0.110048 -0.653967  0.795854  0.813024  0.779535   1.226179   

   cut_Premium  cut_Very Good  cut_Good  ...   color_I   color_J  clarity_IF  \
0    -0.586709       1.853887  -0.31549  ... -0.334127 -0.233613   -0.186791   
1    -0.586709       1.853887  -0.31549  ... -0.334127 -0.233613   -0.186791   
2     1.704423      -0.539407  -0.31549  ... -0.334127 -0.233613   -0.186791   
3     1.704423      -0.539407  -0.31549  ...  2.992871 -0.233613   -0.186791   
4    -0.586709      -0.539407  -0.31549  ...  2.992871 -0.233613   -0.186791   

   clarity_VVS1  clarity_VVS2  clarity_VS1  clarity_VS2  clari

#### model fitten
We gebruiken dus de (gestandaardiseerde) train data om een model te fitten. Dit is, zoals je ziet, een eenvoudig regeltje code. We geven het model een naam (`svr`) zodat we het later weer opnieuw kunnen aanroepen / gebruiken.

In [9]:
# Met model fitten default van package
svr = svm.SVR()
#Fit the regressor object to the dataset.
svr.fit(train_X_scaled,train_y)

SVR()

#### Model weergeven
De mogelijkheden voor het weergeven van (kenmerken van) een gevonden SVM zijn wat beperkt. Bijvoorbeeld het *aantal* benodigde support vectors zegt iets. Hoe meer, hoe complexer het model, hoe groter de kans op overfitting op de trainingsdata.

In [10]:
n_supp_vecs = svr.n_support_
print("Aantal support vectoren is: ", n_supp_vecs )

Aantal support vectoren is:  [37755]


Dit is héél erg veel, namelijk bijna precies het aantal train samples, met andere woorden: (nagenoeg) álle traindata wordt gebruikt als support vector. 

#### Modelscore op train-/test dataset
We zien hieronder dat er sprake is van een (enigszins) overfit model. 

In [11]:
predicted_y_tr = svr.predict(train_X_scaled)
predicted_y = svr.predict(test_X_scaled)
from sklearn import metrics

#RMSE train
print("DE RMSE op de trainset = ", metrics.root_mean_squared_error(train_y, predicted_y_tr))

#RMSE test
print("DE RMSE op de testset = ", metrics.root_mean_squared_error(test_y, predicted_y))

DE RMSE op de trainset =  3303.2454522171
DE RMSE op de testset =  3330.764799493368


C:\Users\kesjs\Anaconda3\envs\BP3.4.2024\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\kesjs\Anaconda3\envs\BP3.4.2024\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


#### Epsilon aanpassen
Om het effect te zien van een andere waarde van `epsilon` fitten we nogmaals een model met een flink grotere waarde voor epsilon (Zie ook bijv. [deze uitleg](https://stats.stackexchange.com/questions/259018/meaning-of-epsilon-in-svm-regression)):

In [13]:
# Met model fitten default van package
svr2 = svm.SVR(epsilon = 100) #in plaats van de default epsilon = 0.1
#Fit the regressor object to the dataset.
svr2.fit(train_X_scaled,train_y)

SVR(epsilon=100)

#### Model weergeven
Het aantal support vectoren is (relatief beperkt) afgenomen:

In [14]:
n_supp_vecs = svr2.n_support_
print("Aantal support vectoren is: ", n_supp_vecs )

Aantal support vectoren is:  [34768]


#### Modelscore op train-/test dataset
Nogmaals de train- en test performance:

In [16]:
predicted_y_tr2 = svr2.predict(train_X_scaled)
predicted_y2 = svr2.predict(test_X_scaled)
from sklearn import metrics

#RMSE train
print("DE RMSE op de trainset = ", metrics.root_mean_squared_error(train_y, predicted_y_tr2))

#RMSE test
print("DE RMSE op de testset = ", metrics.root_mean_squared_error(test_y, predicted_y2))

DE RMSE op de trainset =  3302.232974530879
DE RMSE op de testset =  3329.7241133909292


Geen echt spectaculaire verbetering. 

### 6. Tuning hyperparameters
We hebben onze boom uit het voorbeeld nu gefit voor een model dat de default hyperparameters meegekregen heeft. Verderop onderzoeken we uitkomsten bij andere keuzen van de hyperparameters. 

Het model (ook het tweede) is zo te zien wat overfit op de trainingsdata. Dit hoeft niet per se een probleem te zijn, maar het is wel van belang om te onderzoeken of het model met ándere hyperparameters wellicht betere scoreresultaten behaalt. Gebruikelijk is dit te doen in een K-fold Cross Validatie, zie hiervoor *Tech report 6: K-fold crossvalidatie* uit deze serie. 

Parameters om sowieso te onderzoeken zijn `C`, `epsilon`, `kernel` (kies bijvoorbeeld `'poly'` en `'rbf'`) en dan  daarbij `degree` (bij `'poly'`) respectievelijk `gamma` (bij `'rbf'`) een waardebereik telaten doorlopen. 

#### hyperparameters
Nu gaan we ons bij wijze van voorbeeld richten op een aantal *hyperparameters* die we willen kiezen om wellicht tot beter modelprestaties te komen:  
1. De Regularization parameter van ons model, deze bepaalt de mate van misclassificatie die we willen vermijden: `C = .. .`  
2. De kernel, deze bepaalt de wiskundige transformatie van de data: `kernel = ...`  zetten we nu **vast** op `'rbf'`. Voor een grondige verkenning zou ook het doorlopen van andere opties zijn aan te raden, maar dit model met deze dataset trainen erg lang en voor dit Tech Report is dit slechts ter illustratie bedoeld. 
3. Bij elke kernel hoort weer een eigen parameter, we doorlopen voor de `kernel = 'rbf'` de parameter `gamma`

#### Grid search
Hieronder zien we hoe we een *grid*  definieren van modelparameters die we doorlopen en bij elke combinatie wordt een score bepaald o.b.v. een Cross-Validatie (k = 4). ook opm de traintijd niet te zeet op te blazen in dit Tech Report is gekozen voor een erg klein grid en zul je in de parktijk breder en langer gaan zoeken.

We gebruiken de geschaalde traindata.

<div class="alert alert-block alert-danger">
    <h1>Let op - deze grid search duurt lang!!</h1>
</div>

**Let op!** Onderstaande gridsearch duurt erg lang! 

In [17]:
# Hyper parameters range om te doorlopen
# Dit is slechts een voorbeeld, ook de ándere modelparameters kunnen hierin worden betrokken
import numpy as np

parameters_rbf={"kernel" : ["rbf"],               # NB: dit is de default
            "gamma" : 10. ** np.arange(-2, 1),
               "C" : 10. ** np.arange(0, 3)
           }

In [18]:
# Importeer vanuit scikit learn package de tooling:
from sklearn.model_selection import GridSearchCV

We starten hieronder de grid search. We maken tweemaal een `tuning_model` waarin alle resultaten van de getrainde modellen op alle gridpunten zullen worden bewaard. Zie het *Tech report 6 K-fold crossvalidatie en Grid Search* in deze serie voor meer uitleg en voorbeelden.

Eerst volgt een definitie van de grid search. We zetten (om de output in dit notebook niet te laten exploderen) `verbose = 1`, maar dit kun je zélf eventueel aanpassen.

In [19]:
svr_grid_rbf = svm.SVR()
tuning_model_rbf = GridSearchCV(estimator = svr_grid_rbf, param_grid = parameters_rbf,scoring='neg_mean_squared_error',cv=3,verbose=1)

Hieronder starten we de daarwerkelijke evaluatie, deze stap kan in sommigen gevallen best lang duren!

In [20]:
tuning_model_rbf.fit(train_X_scaled,train_y)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


GridSearchCV(cv=3, estimator=SVR(),
             param_grid={'C': array([  1.,  10., 100.]),
                         'gamma': array([0.01, 0.1 , 1.  ]),
                         'kernel': ['rbf']},
             scoring='neg_mean_squared_error', verbose=1)

Een optimum voor radiale kernel werd gevonden voor:

In [21]:
print("Beste model: ", tuning_model_rbf.best_estimator_)
print("Beste score:  MSE = ", np.sqrt(-tuning_model_rbf.best_score_))
print("Beste parameter combi: ", tuning_model_rbf.best_params_)

Beste model:  SVR(C=100.0, gamma=0.01)
Beste score:  MSE =  988.6399786666015
Beste parameter combi:  {'C': 100.0, 'gamma': 0.01, 'kernel': 'rbf'}


Het doorlopen grid is nogal klein en met relatief grote stappen geweest, er is voldoende ruimte dit verder te finetunen, dit laten we aan jullie. 

### 7. Trainen model met optimale parameterwaarden
We trainen hieronder het model nogmaals, maar dan op de gehele train dataset en mét de gevonden optimale parameterwaarden. 

**Zie echter ook sectie *7a*  hieronder!**

In [23]:
#Definitie
svr_opt = svm.SVR(C=100.0, gamma=0.01) #kernel default = 'rbf'
#fitten
svr_opt.fit(train_X_scaled,train_y)


SVR(C=100.0, gamma=0.01)

We maken voorspelling op de train data en evalueren deze:

In [24]:
predicted_y_tr = svr_opt.predict(train_X_scaled)
from sklearn import metrics

#accuracy train
print("De RMSE op de trainset = ", metrics.root_mean_squared_error(train_y, predicted_y_tr))

De RMSE op de trainset =  903.4329264151967


### 7a Opnieuw fitten niet nodig
Bovenstaande stap kon bij nader inzien ook door *direct* het reeds beschikbare model `tuning_model_rbf.best_estimator_` te gebruiken, dit levert `GridSearchCV()` al vanzelf! 

In [22]:
# Opnieuw fitten overbodig?
model_opt_grid = tuning_model_rbf.best_estimator_ #getrainde model uit gridsearch
predicted_y_tr_alt = model_opt_grid.predict(train_X_scaled)

from sklearn import metrics

#accuracy train
print("De RMSE_alt op de trainset = ", metrics.root_mean_squared_error(train_y, predicted_y_tr_alt))

De RMSE_alt op de trainset =  903.4329264151967


### 8. Voorspellingen en performance op testset
Hieronder genereren we voorspellingen van y op de testset. Tevens produceren we een modelscore. Voor meer toelichting daarover verwijzen we naar *Tech report 3: performance metrics regressie en classificatie*.

We zien dat de mate van overfitting kleiner is, maar de prestatie op test *niet* beter dan bij het *ongelimiteerde zeer complexe model*. Dat kan gebeuren en is bijvoorbeeld een gevolg van de toevallige samenstelling van de *test data*. NB: wat cf. *CRISP-DM* nu **niet** is toegestaan is dat deze observatie je doet besluiten *opnieuw* aan de model architectuur (de hyper  parameters) te tunen! 

In [ ]:
predicted_y = svr_opt.predict(test_X_scaled)

#accuracy test
print("De RMSE op de testset = ", metrics.root_mean_squared_error(test_y, predicted_y))

De prestaties op de traindata zijn ten opzichte van het default model wel iets toegenomen, maar op test ongewijzigd gebleven t.o.v. het default model.